In [52]:
from astropy.io import fits
import numpy as np
import pandas as pd

## Antenna Layout

* fitsrec from numpy.record
* 

Keys
* ANNAME
* STABXYZ
* ORBPARM
* NOSTA
* MNTSTA
* STAXOF
* DIAMETER
* BEAMFWHM (num_ant, 4)
* POLTYA
* POLAA
* POLCALA
* POLTYB
* POLAB
* POLCALB

In [ ]:
f = fits.open("../../test.uvf")

In [326]:
f[2].header

XTENSION= 'BINTABLE'           / Extension type                                 
BITPIX  =                    8 / Binary data                                    
NAXIS   =                    2 / Table is a matrix                              
NAXIS1  =                  116 / Width of table in bytes                        
NAXIS2  =                    1 / Number of entries in table                     
PCOUNT  =                    0 / Random parameter count                         
GCOUNT  =                    1 / Group count                                    
TFIELDS =                    6 / Number of fields in each row                   
EXTNAME = 'AIPS FQ '           / AIPS table file                                
EXTVER  =                    1 / Version number of table                        
TFORM1  = '1J      '           / FORTRAN format of field  1                     
TTYPE1  = 'FRQSEL          '   / Type (heading) of field  1                     
TUNIT1  = '        '        

In [325]:
f[2].data

FITS_rec([(1, [0.00e+00, 8.00e+07, 1.44e+08, 2.08e+08], [64000000., 64000000., 64000000., 64000000.], [64000000., 64000000., 64000000., 64000000.], [1, 1, 1, 1], '')],
         dtype=(numpy.record, [('FRQSEL', '>i4'), ('IF FREQ', '>f8', (4,)), ('CH WIDTH', '>f4', (4,)), ('TOTAL BANDWIDTH', '>f4', (4,)), ('SIDEBAND', '>i4', (4,)), ('RXCODE', 'S32')]))

In [327]:
f[2].data.columns

ColDefs(
    name = 'FRQSEL'; format = '1J'
    name = 'IF FREQ'; format = '4D'; unit = 'HZ'
    name = 'CH WIDTH'; format = '4E'; unit = 'HZ'
    name = 'TOTAL BANDWIDTH'; format = '4E'; unit = 'HZ'
    name = 'SIDEBAND'; format = '4J'
    name = 'RXCODE'; format = '32A'
)

In [310]:
def create_antenna_hdu(layout_txt):
    array = pd.read_csv(layout_txt, sep=" ")
    
    ANNAME = np.chararray(len(array), itemsize=8, unicode=True)
    ANNAME[:] = array["station_name"].values
    col1 = fits.Column(name='ANNAME', format='8A', array=ANNAME)
    
    STABXYZ = np.array([array["X"], array["Y"], array["Z"]], dtype=">f8").T
    col2 = fits.Column(name='STABXYZ', format='3D', unit="METERS", array=STABXYZ)

    ORBPARM = np.array([], dtype=">f8")
    col3 = fits.Column(name='ORBPARM', format='0D', unit=" ", array=ORBPARM)

    NOSTA = np.arange(len(array), dtype=">i4")
    col4 = fits.Column(name='NOSTA', format='1J', unit=" ", array=NOSTA)

    MNTSTA = np.zeros(len(array), dtype=">i4")
    col5 = fits.Column(name='MNTSTA', format='1J', unit=" ", array=MNTSTA)

    STAXOF = np.zeros(len(array), dtype=">f4")
    col6 = fits.Column(name='STAXOF', format='1E', unit="METERS", array=STAXOF)

    DIAMETER = np.array(array["dish_dia"].values, dtype=">f4")
    col7 = fits.Column(name='DIAMETER', format='1E', unit="METERS", array=DIAMETER)

    BEAMFWHM = np.zeros((len(array), 4), dtype=">f4")
    col8 = fits.Column(name='BEAMFWHM', format='4E', unit="DEGR/M", array=BEAMFWHM)

    POLTYA = np.chararray(len(array), itemsize=1, unicode=True)
    POLTYA[:] = "R"
    col9 = fits.Column(name='POLTYA', format='1A', unit=" ", array=POLTYA)

    POLAA = np.zeros(len(array), dtype=">f4")
    col10 = fits.Column(name='POLAA', format='1E', unit="DEGREES", array=POLAA)

    POLCALA = np.zeros((len(array), 8), dtype=">f4")
    col11 = fits.Column(name='POLCALA', format='8E', unit=" ", array=POLCALA)

    POLTYB = np.chararray(len(array), itemsize=1, unicode=True)
    POLTYB[:] = "L"
    col12 = fits.Column(name='POLTYB', format='1A', unit=" ", array=POLTYB)

    POLAB = np.zeros(len(array), dtype=">f4")
    col13 = fits.Column(name='POLAB', format='1E', unit="DEGREES", array=POLAB)

    POLCALB = np.zeros((len(array), 8), dtype=">f4")
    col14 = fits.Column(name='POLCALB', format='8E', unit=" ", array=POLCALB)

    coldefs_ant = fits.ColDefs([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12, col13, col14])
    hdu_ant = fits.BinTableHDU.from_columns(coldefs_ant)
    
    # add additional keywords
    hdu_ant.header["EXTNAME"] = ("AIPS AN", "AIPS table file")
    hdu_ant.header["EXTVER"] = (1, "Version number of table")
    hdu_ant.header["ARRAYX"] = (0, "x coordinate of array center (meters)")
    hdu_ant.header["ARRAYY"] = (0, "y coordinate of array center (meters)")
    hdu_ant.header["ARRAYZ"] = (0, "z coordinate of array center (meters)")
    hdu_ant.header["GSTIA0"] = (119.85, "GST at 0h on reference date (degrees)") # how to calculate??
    hdu_ant.header["DEGPDY"] = (360.98564497329994, "Earth's rotation rate (degrees/day)")
    hdu_ant.header["FREQ"] = (227297000000.0, "Reference frequency (Hz)") # read from simulations
    hdu_ant.header["RDATE"] = ("2015-04-08T00:00:00.0", "Reference date") # where do we find this?
    hdu_ant.header["POLARX"] = (0, "x coordinate of North Pole (arc seconds)") # missing, 0.10819000005722046 MOJAVE
    hdu_ant.header["POLARY"] = (0, "y coordinate of North Pole (arc seconds)") # missing 0.28815001249313354 MOJAVE
    hdu_ant.header["UT1UTC"] = (0, "UT1 - UTC (sec)") # missing
    hdu_ant.header["DATUTC"] = (0, "time system - UTC (sec)") # missing
    hdu_ant.header["TIMSYS"] = ("UTC", "Time system")
    hdu_ant.header["ARRNAM"] = ("EHT", "Array name") # read from layout file
    hdu_ant.header["XYZHAND"] = ("RIGHT", "Handedness of station coordinates")
    hdu_ant.header["FRAME"] = ("????", "Coordinate frame")
    hdu_ant.header["NUMORB"] = (0, "Number orbital parameters in table (n orb)")
    hdu_ant.header["NOPCAL"] = (2, "Number of polarization calibration values / IF(n pcal)")
    hdu_ant.header["NO_IF"] = (4, "Number IFs (n IF)")
    hdu_ant.header["FREQID"] = (-1, "Frequency setup number")
    hdu_ant.header["IATUTC"] = (33, "No one knows.....") # how to calculate??
    hdu_ant.header["POLTYPE"] = (" ", "Type of polarization calibration")

    # add comments
    hdu_ant.header.comments["TTYPE1"] = "Antenna name"
    hdu_ant.header.comments["TTYPE2"] = "Antenna station coordinates (x, y, z)"
    hdu_ant.header.comments["TTYPE3"] = "Orbital parameters"
    hdu_ant.header.comments["TTYPE4"] = "Antenna number"
    hdu_ant.header.comments["TTYPE5"] = "Mount type"
    hdu_ant.header.comments["TTYPE6"] = "Axis offset"
    hdu_ant.header.comments["TTYPE7"] = "Antenna diameter"
    hdu_ant.header.comments["TTYPE8"] = "Antenna beam FWHM"
    hdu_ant.header.comments["TTYPE9"] = "R, L, feed A"
    hdu_ant.header.comments["TTYPE10"] = "Position angle feed A"
    hdu_ant.header.comments["TTYPE11"] = "Calibration parameters feed A"
    hdu_ant.header.comments["TTYPE12"] = "R, L, polarization 2"
    hdu_ant.header.comments["TTYPE13"] = "Position angle feed B"
    hdu_ant.header.comments["TTYPE14"] = "Calibration parameters feed B"
    
    return hdu_ant

In [311]:
path = "../vipy/layouts/eht.txt"
ant_hdu = create_antenna_hdu(path)
ant_hdu.data

FITS_rec([('ALMA50', [ 2225037.1851, -5441199.162 , -2479303.4629]),
          ('SMTO', [-1828796.2   , -5054406.8   ,  3427865.2   ]),
          ('LMT', [ -768713.9637, -5988541.7982,  2063275.9472]),
          ('Hawaii8', [-5464523.4   , -2493147.08  ,  2150611.75  ]),
          ('PV', [ 5088967.9   ,  -301681.6   ,  3825015.8   ]),
          ('PdBI', [ 4523998.4   ,   468045.24  ,  4460309.76  ]),
          ('SPT', [       0.    ,        0.    , -6359587.3   ]),
          ('GLT', [ 1500692.    , -1191735.    ,  6066409.    ])],
         dtype=(numpy.record, [('ANNAME', 'S8'), ('STABXYZ', '<f8', (3,)), ('ORBPARM', '<f8', (0,)), ('NOSTA', '<i4'), ('MNTSTA', '<i4'), ('STAXOF', '<f4'), ('DIAMETER', '<f4'), ('BEAMFWHM', '<f4', (4,)), ('POLTYA', 'S1'), ('POLAA', '<f4'), ('POLCALA', '<f4', (8,)), ('POLTYB', 'S1'), ('POLAB', '<f4'), ('POLCALB', '<f4', (8,))]))

In [322]:
ant_hdu.data["POLAB"]

array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [296]:
cel = fits.open("../../celestial-01-05.uvfits")

In [319]:
cel[1].data

FITS_rec([('ALMA50', [ 2225037.1851, -5441199.162 , -2479303.4629], 0., 1, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('SMTO', [-1828796.2   , -5054406.8   ,  3427865.2   ], 0., 2, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('LMT', [ -768713.9637, -5988541.7982,  2063275.9472], 0., 3, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('Hawaii8', [-5464523.4   , -2493147.08  ,  2150611.75  ], 0., 4, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('PV', [ 5088967.9   ,  -301681.6   ,  3825015.8   ], 0., 5, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('PdBI', [ 4523998.4   ,   468045.24  ,  4460309.76  ], 0., 6, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('SPT', [       0.    ,        0.    , -6359587.3   ], 0., 7, 0, 0., 'R', 0., [0., 0., 0.], 'L', 90., [0., 0., 0.]),
          ('GLT', [ 1500692.    , -1191735.    ,  6066409.    ], 0., 8, 0, 0., 'R', 0., [0., 0., 0.], '

In [324]:
f[3].header

XTENSION= 'BINTABLE'           / Extension type                                 
BITPIX  =                    8 / Binary data                                    
NAXIS   =                    2 / Table is a matrix                              
NAXIS1  =                  138 / Width of table in bytes                        
NAXIS2  =                   10 / Number of entries in table                     
PCOUNT  =                    0 / Random parameter count                         
GCOUNT  =                    1 / Group count                                    
TFIELDS =                   14 / Number of fields in each row                   
EXTNAME = 'AIPS AN '           / AIPS table file                                
EXTVER  =                    1 / Version number of table                        
TFORM1  = '8A      '           / FORTRAN format of field  1                     
TTYPE1  = 'ANNAME          '   / Type (heading) of field  1                     
TUNIT1  = '        '        